In [1]:
import torch
from src.models.blm.pl_dataloader import TinyStoriesDataloader
from src.models.blm.pl_training import Transformer

Seed set to 123
Seed set to 123


In [8]:
BASE_URL = "/home/pranav-pc/projects/OpenTransformer/multiformer"
MODEL_CHECKPOINT_PATH = "/home/pranav-pc/projects/model-registry/blm-medium/last.ckpt"
data_path_train = BASE_URL + "/data/interim/TinyStories_train_65>tk>512.hf"
data_path_val = BASE_URL + "/data/interim/TinyStories_val_65>tk>512.hf"
tokenizer_path = BASE_URL + "/tokenizer_checkpoints"


batch_size = 32
num_workers = 26
ds = TinyStoriesDataloader(
    data_path_train, data_path_val, tokenizer_path, batch_size, num_workers, subset_ratio=1
)

In [9]:
model = Transformer.load_from_checkpoint(MODEL_CHECKPOINT_PATH)
model = torch.compile(model, dynamic=True)

#### Inference
model.eval()
model = model.cuda()

import os

os.environ["WANDB_DISABLED"] = "true"

In [10]:
text = "Lili wanted to watch either a movie or a cartoon. Her mother didn’t let her watch a cartoon so instead she"


def predict(text, max_new_tokens=450, temperature=0.8, top_k=3, conditional_break=[13, 13, 1]):
    tokens = torch.LongTensor(ds.tokenizer.encode_as_ids(text))[:-1].to("cuda:0").view(1, -1)
    # print('tokens',ds.tokenizer.encode(text,out_type=str)[:-1])
    print(
        ds.tokenizer.decode_ids(
            model.predict_step(
                tokens,
                None,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_k=top_k,
                conditional_break=conditional_break,
            )[0].tolist()
        )
    )


predict(text)

Lili wanted to watch either a movie or a cartoon. Her mother didn’t let her watch a cartoon so instead she decided to go to the theater. When she got there, she saw a very special cartoon. It was about a princess who was so spoiled!

Lili was very excited. She asked her mother if she could watch the cartoon, but her mother said no. Lili was very disappointed.

Then Lili's mother said, "If you want to watch the cartoon, you can watch the cartoon. But you have to be careful not to get too excited."

So Lili went to the store and bought a cartoon. She watched it all the way home. She was so excited that she had a cartoon to watch. She was so happy that she could finally watch the cartoon. She was so glad she had listened to her mother and was so spoiled by the cartoon.




In [13]:
texts = [
    "I loved the movie. It was great!",
    "The food was terrible.",
    "The weather is okay.",
]
sentiments = ["positive", "negative", "neutral"]

In [15]:
# predict(texts)

In [9]:
# Tokenize the text samples
encoded_texts = ds.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Extract the input IDs and attention masks
input_ids = encoded_texts["input_ids"]
attention_mask = encoded_texts["attention_mask"]

# Convert the sentiment labels to numerical form
sentiment_labels = [sentiments.index(sentiment) for sentiment in sentiments]

In [11]:
input_ids

tensor([[ 101, 1045, 3866, 1996, 3185, 1012, 2009, 2001, 2307,  999,  102],
        [ 101, 1996, 2833, 2001, 6659, 1012,  102,    0,    0,    0,    0],
        [ 101, 1996, 4633, 2003, 3100, 1012,  102,    0,    0,    0,    0]])

In [10]:
attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])

In [16]:
import torch.nn as nn

classification_head = nn.Linear(model.config.hidden_size, 3)

# Replace the pre-trained model's classification head with our custom head
model.classifier = classification_head

import torch.optim as optim

# Define the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Fine-tune the model
num_epochs = 3
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(
        input_ids, attention_mask=attention_mask, labels=torch.tensor(sentiment_labels)
    )
    loss = outputs.loss
    loss.backward()
    optimizer.step()

RuntimeError: shape '[-1, 2]' is invalid for input of size 9

In [ ]:
# Incontext learning

### GLUE (General Language Understanding Evaluation)


**Single-Sentence Tasks**

1. CoLA (Corpus of Linguistic Acceptability)
- Goal: determine if a sentence is grammatically correct or not.
- Dataset: it consists of English acceptability judgments drawn from books and journal articles. Each example is a sequence of words annotated with whether it is a correct grammatical English sentence or not.
2. SST-2 (Stanford Sentiment Treebank)
- Goal: determine if the sentence has a positive or negative sentiment.
- Dataset: it consists of sentences from movie reviews and binary human annotations of their sentiment.


**Similarity and Paraphrase Tasks**

3. MRPC (Microsoft Research Paraphrase Corpus)
- Goal: determine if two sentences are paraphrases from one another.
- Dataset: it’s a corpus of sentence pairs automatically extracted from online news sources, with human annotations indicating whether the sentences in the pair are semantically equivalent (i.e. paraphrases).
4. QQP (Quora Question Pairs)
- Goal: determine if two questions are semantically equivalent or not.
- Dataset: it’s a collection of question pairs from the community question-answering website Quora, with human annotations indicating whether the questions in the pair are actually the same question.
5. STS-B (Semantic Textual Similarity Benchmark)
- Goal: determine the similarity of two sentences with a score from one to five.
- Dataset: it’s a collection of sentence pairs drawn from news headlines, video and image captions, and natural language inference data. Each pair is annotated by humans with a similarity score from one to five.


**Inference Tasks**

6. MNLI (Multi-Genre Natural Language Inference)
- Goal: determine if a sentence entails, contradicts, or is unrelated to another sentence.
- Dataset: it’s a crowdsourced collection of sentence pairs with textual entailment annotations. The premise sentences are gathered from ten different sources, including transcribed speech, fiction, and government reports. The dataset has two test sets: a matched (in-domain) and mismatched (cross-domain) test set. The scores on the matched and mismatched test sets are then averaged together to give the final score on the MNLI task.
7. QNLI (Question-answering Natural Language Inference)
- Goal: determine if the answer to a question is contained in a second sentence or not.
- Dataset: it’s a question-answering dataset consisting of question-paragraph pairs, where one of the sentences in the paragraph (drawn from Wikipedia) contains the answer to the corresponding question (written by an annotator).
8. RTE (Recognizing Textual Entailment)
- Goal: determine if a sentence entails a given hypothesis or not.
- Dataset: it is a combination of data from annual textual entailment challenges (i.e. from RTE1, RTE2, RTE3, and RTE5). Examples are constructed based on news and Wikipedia text.
9. WNLI (Winograd Natural Language Inference)
- Goal: determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not.
- Dataset: this dataset is built from the Winograd Schema Challenge dataset, where it’s a reading comprehension task in which a system must read a sentence with a pronoun and select the referent of that pronoun from a list of choices. To convert the problem into sentence pair classification, the authors of the benchmark construct sentence pairs by replacing the ambiguous pronoun with each possible referent. The examples are manually constructed to foil simple statistical methods: each one is contingent on contextual information provided by a single word or phrase in the sentence.


In [18]:
import lightning.pytorch as pl
from datasets import load_dataset

In [24]:
from dataclasses import dataclass

In [25]:
@dataclass
class GLUEConfig:
    model_path: str
    tokenizer_path: str
    task_name: str = "mrpc"
    max_seq_len: int = 512
    train_batch_size: int = 64
    eval_batch_size: int = 128

In [26]:
class GLUEDataModule(pl.LightningDataModule):
    task_text_field_map = {
        "cola": ["sentence"],
        "sst2": ["sentence"],
        "mrpc": ["sentence1", "sentence2"],
        "qqp": ["question1", "question2"],
        "stsb": ["sentence1", "sentence2"],
        "mnli": ["premise", "hypothesis"],
        "qnli": ["question", "sentence"],
        "rte": ["sentence1", "sentence2"],
        "wnli": ["sentence1", "sentence2"],
        "ax": ["premise", "hypothesis"],
    }

    glue_task_num_labels = {
        "cola": 2,
        "sst2": 2,
        "mrpc": 2,
        "qqp": 2,
        "stsb": 1,
        "mnli": 3,
        "qnli": 2,
        "rte": 2,
        "wnli": 2,
        "ax": 3,
    }

    loader_columns = [
        "datasets_idx",
        "input_ids",
        "token_type_ids",
        "attention_mask",
        "start_positions",
        "end_positions",
        "labels",
    ]

    def __init__(self, args):
        super().__init__()
        self.args = args

        self.column_name = self.task_text_field_map[args.task_name]
        self.label_count = self.glue_task_num_labels[args.task_name]
        self.tokenizer = self._load_tokenizer(self.args.tokenizer_path)

    @staticmethod
    def _load_tokenizer(tokenizer_path):
        from src.tokenize.tokenizer import Tokenizer

        return Tokenizer(tokenizer_path)

    def setup(self, stage: str):
        self.dataset = load_dataset("glue", self.args.task_name)

In [19]:
dataset = load_dataset("glue", "stsb", cache_dir="eval-data/")

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [29]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1379
    })
})

In [33]:
example = dataset["train"][0]
list(zip(example["sentence1"], example["sentence1"]))

[('A', 'A'),
 (' ', ' '),
 ('p', 'p'),
 ('l', 'l'),
 ('a', 'a'),
 ('n', 'n'),
 ('e', 'e'),
 (' ', ' '),
 ('i', 'i'),
 ('s', 's'),
 (' ', ' '),
 ('t', 't'),
 ('a', 'a'),
 ('k', 'k'),
 ('i', 'i'),
 ('n', 'n'),
 ('g', 'g'),
 (' ', ' '),
 ('o', 'o'),
 ('f', 'f'),
 ('f', 'f'),
 ('.', '.')]

In [37]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [41]:
tokenized_datasets["train"][6]

{'sentence1': 'The Nasdaq had a weekly gain of 17.27 , or 1.2 percent , closing at 1,520.15 on Friday .',
 'sentence2': 'The tech-laced Nasdaq Composite .IXIC rallied 30.46 points , or 2.04 percent , to 1,520.15 .',
 'label': 0,
 'idx': 6,
 'input_ids': [101,
  1996,
  17235,
  2850,
  4160,
  2018,
  1037,
  4882,
  5114,
  1997,
  2459,
  1012,
  2676,
  1010,
  2030,
  1015,
  1012,
  1016,
  3867,
  1010,
  5494,
  2012,
  1015,
  1010,
  19611,
  1012,
  2321,
  2006,
  5958,
  1012,
  102,
  1996,
  6627,
  1011,
  17958,
  17235,
  2850,
  4160,
  12490,
  1012,
  11814,
  2594,
  24356,
  2382,
  1012,
  4805,
  2685,
  1010,
  2030,
  1016,
  1012,
  5840,
  3867,
  1010,
  2000,
  1015,
  1010,
  19611,
  1012,
  2321,
  1012,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  